In [ ]:
## Requirements:
#Python 3.9.5>
#pandas

In [40]:
!python --version

Python 3.9.5


# 1 Theory

## Table of content for theory: 

1.  N-gram concept usecase
        1.1. n-gram
        1.1. computing "joint probability with conditional probability" 
        1.3. Markof assumption 
2. How efficiently store N-grams ?
3. How efficiently find all 1,2,...,n-grams ?

4.  N-gram as a Language Model
        4.1. Blank Completion ( text __  / text __ text )
        4.2. Finding Probability of occurence of a sentence
5. Possible drawbacks of n-grams

# 2 Hands-on

## Table of content for hands-on: 

1.  Implementation of functions
2.  Driver code
    1. Question 1 and 2 answers
    2. Question3 
    3. Question4
    4. Question5


### Implementation of functions

In [54]:
# Assumption: Corpus can fit in RAM and we don't need to read and store it line by line+

def load_data(address):
    # load strings and process them finally -->
    # return a list of list containing tokens
    sentences = []
    with open(address, "r",encoding="utf-8") as file:
        # := is a new feature in py3.8 called walrus operator 
        # https://docs.python.org/3/whatsnew/3.8.html#assignment-expressions
        while line := file.readline():
            processed = preprocessing(line) 
            if not processed == -1:
                # note: appending a list is amortized O(1)
                # so, we don't need to be worried about it
                sentences.append(processed)
    return sentences

def preprocessing(string):
    res = string.rstrip().lstrip().split()
    return res if len(res)>3 else -1




def accum_ngram(corp , n_gram = 5):
    
    # Python dictionary has dynamic hashing and can access each key's value in O(1) time.
    # It, however, needs O(n) space to store n strings. "Trie" is an attempt to reduce 
    # this space complexity
    
    # accumulated n-grams 
    # by accumulated I mean we compute all n-grams from 1 up to n
    # Result will be stored and returned as a trie, implemented with python dictionary
    
    n_grams = {}
    n_grams['#'] = 0 
    for stc in corp:
        n = len(stc)
        n_grams['#'] += n # update total words count
        for token_idx in range(n):
            dict_ptr = n_grams
            for next_token_idx in range(token_idx,n):
                if next_token_idx - token_idx >= n_gram:
                    break
                #print(f'token_idx:{token_idx}, next_token_idx:{next_token_idx},the word:{stc[next_token_idx]}')
                if not stc[next_token_idx] in dict_ptr:
                    dict_ptr[stc[next_token_idx]] = {'#':0}
                    
                dict_ptr[stc[next_token_idx]]['#'] = dict_ptr[stc[next_token_idx]]['#'] + 1
                dict_ptr = dict_ptr[stc[next_token_idx]]
                
            
    return n_grams



In [55]:
def count_finder(sentence, n_grams_dict, j, i):
    # find both numerator and numerator 
    ptr_dict = n_grams_dict
    
    for k in range(j,i):
        ptr_dict = ptr_dict[sentence[k]]

    return ptr_dict[sentence[i]]['#'], ptr_dict['#']

def occurance_probability(sentence, n_grams_dict, n_gram = 2):
    if n_gram ==1:
        prob = 1
        for token in sentence:
            prob *= n_grams_dict[token]['#']
        prob *= (1/(n_grams_dict['#']**len(sentence)))
        return prob
    
    # n_gram is the numeber of previous words each conditional probability depends on
    # sentence: list of tokens
    
    # handeling the first fraction separately
    prob = n_grams_dict[sentence[0]]['#']/n_grams_dict['#'] 
    
    for i in range(1,len(sentence)):
        j = i - (n_gram -1) if (i-n_gram>=0) else 0
        numerator, denumerator =count_finder(sentence=sentence, n_grams_dict=n_grams_dict, j = j, i = i)
        prob *= (numerator/denumerator)
        
    return prob

In [56]:
def dfs(ptr_dict, current_str, remained_depth, counts_ls, terms_ls):
    if remained_depth == 1:
        keys = list(ptr_dict.keys())
        keys.pop(0) # first key is '#'
        for key in keys:
            s = current_str + ' ' + key # we gotta remove first ' '
            terms_ls.append(s[1:])
            counts_ls.append(ptr_dict[key]['#'])
    else:
        keys = list(ptr_dict.keys())
        keys.pop(0)
        for key in keys:
            dfs(ptr_dict= ptr_dict[key],current_str = current_str+' '+key,
                remained_depth =remained_depth-1,counts_ls = counts_ls, terms_ls=terms_ls)
        
            

In [57]:
# Visualize different n_grams with tables

def visual(n_grams_dict, n_gram ):
    # n_gram dicts contains all n grams in a trie data structure
    # we convert trie structure into pandas df
    import pandas as pd
    counts = []
    terms = []
    dfs(n_grams_dict, current_str = '', remained_depth = n_gram,
            counts_ls = counts, terms_ls = terms)
    
    # tabularizing
    n  = sum(counts)
    size = len(terms)
    df = pd.DataFrame({'terms':terms})
    df['counts'] = counts
    df.sort_values(by=['counts'], inplace=True, ascending=False,ignore_index=True)
    df['freq'] = df['counts'].apply(lambda x: x/n)
    df['freq_index'] = df['freq'] * [i for i in range(1,size+1)]
    return df
    
    
            

In [109]:
def question3handler(string):
    grams_prob =[-1,-1,-1] #-1 means "error raised"
    s0 = preprocessing(string)
    for i in range(0,3):
        try:
            grams_prob[i] = occurance_probability(s0, accum_five_gram, i+1)
        except:
            pass
    print(f'For this sentence: {string} we have: \n1_gram_prob:{grams_prob[0]}\n2_gram_prob:{grams_prob[1]}\n3_gram_prob:{grams_prob[2]}')

In [147]:
def question5handler(string,df):
    indices = []
    s = preprocessing(string)
    last_token =  s[-1]
    n = len(s[-1])
    df_ln = df.shape[0]
    for i in range(df_ln):
        if df['terms'].values[i][:n] == last_token and df['terms'].values[i][n] == ' ':
            indices.append(i)
    print(f'The sentence: {string} can be completed with bigrams in the following way:\n')
    option1 = string + df['terms'][indices[0]][n:] 
    option2 = string + df['terms'][indices[1]][n:]
    print(f'First way for completing: {option1}') 
    print(f'Second way for completing: {option2}\n')
    return df.loc[indices]


### Driver code

In [83]:
loaded_sentences = load_data("train.txt")
loaded_sentences[:10]

[['زانک', 'دل', 'یا', 'اوست', 'یا', 'خود', 'اوست', 'دل'],
 ['عکس', 'هر', 'نقشی', 'نتابد', 'تا', 'ابد'],
 ['پیروز', 'شد', 'از', 'طلعت', 'او', 'دولت', 'و', 'اختر'],
 ['ای', 'تیغ', 'گهردار', 'تو', 'از', 'فتح', 'مرکب'],
 ['باز', 'رویاند', 'گل', 'صباغ', 'را'],
 ['کای', 'بسوزیده', 'برون', 'آ', 'تازه', 'شو'],
 ['سینه', 'شیرین', 'خبر', 'دارد', 'ز', 'خسرو', 'بس', 'بود'],
 ['ناله', 'گردون', 'کفایت', 'باشد', 'از', 'تقدیر', 'بار'],
 ['بریاد', 'بط', 'باده', 'دوانست', 'بهر', 'در'],
 ['شعرش', 'همه', 'ژاژست', 'وکلامش', 'همه', 'یاوه']]

#### Answer to Q1 and Q2

In [76]:
accum_one_gram = accum_ngram(loaded_sentences , n_gram = 1)
accum_two_gram = accum_ngram(loaded_sentences , n_gram = 2)
#accum_three_gram = accum_ngram(loaded_sentences , n_gram = 3)
accum_five_gram = accum_ngram(loaded_sentences , n_gram = 5)


In [159]:
d1 = visual(n_grams_dict = accum_five_gram, n_gram = 1) # x_gram accaptable for any x>=1 
#d.head(20)
d1.head(5)


,terms,counts,freq,freq_index
0,و,50912,0.038950,0.038950
1,از,31178,0.023852,0.047705
2,به,30037,0.022980,0.068939
3,که,26787,0.020493,0.081973
4,در,23525,0.017998,0.089988


In [107]:
d2 = visual(n_grams_dict = accum_five_gram, n_gram = 2) # x_gram accaptable for any x>=1 
#d.head(20)
d2.head(5)

,terms,counts,freq,freq_index
0,از آن,1394,0.001246,0.001246
1,که در,1247,0.001115,0.002229
2,او را,1123,0.001004,0.003011
3,را به,1105,0.000988,0.003951
4,که از,1070,0.000956,0.004782


In [105]:
d5 = visual(n_grams_dict = accum_five_gram, n_gram = 5) # x_gram accaptable for any x>=1 
#d.head(20)
d5.head(5)

,terms,counts,freq,freq_index
0,می سوزم و می سازم,11,0.000020,0.000020
1,خاک و باد و آب,10,0.000018,0.000036
2,ای بی وفا ای پاسبان,10,0.000018,0.000054
3,و باد و آب و,10,0.000018,0.000072
4,خبر ده که تا کجاست,8,0.000014,0.000072


#### Answer to Q3

In [110]:
inp = ["چون تویی آید به زیبایی و شیرینی پسر","دل در این درد و رنج پاره کنیم","ای به آرام تو زمین را سنگ","جان را زند آ باغ صلاهای تعالوا","شاهد و شمع و شراب و مطرب آنجا بهترست","شب است و شمع و شراب و شیرینی"]
for sentence in inp:
    question3handler(sentence)

For this sentence: چون تویی آید به زیبایی و شیرینی پسر we have: 
1_gram_prob:1.5881306963873775e-24
2_gram_prob:2.653444213453881e-19
3_gram_prob:7.589683679349442e-11
For this sentence: دل در این درد و رنج پاره کنیم we have: 
1_gram_prob:2.3266120884334263e-22
2_gram_prob:2.181627522993893e-18
3_gram_prob:2.9968695710428407e-12
For this sentence: ای به آرام تو زمین را سنگ we have: 
1_gram_prob:1.7214192877055589e-18
2_gram_prob:1.8831712377608751e-19
3_gram_prob:1.9030848628816505e-09
For this sentence: جان را زند آ باغ صلاهای تعالوا we have: 
1_gram_prob:7.222886310676774e-27
2_gram_prob:5.342423098525576e-16
3_gram_prob:1.5300802297568475e-07
For this sentence: شاهد و شمع و شراب و مطرب آنجا بهترست we have: 
1_gram_prob:2.261582778132732e-28
2_gram_prob:5.72150764844012e-22
3_gram_prob:2.709236115708343e-11
For this sentence: شب است و شمع و شراب و شیرینی we have: 
1_gram_prob:1.6337996873326272e-21
2_gram_prob:3.3462669538796835e-19
3_gram_prob:-1


#### Answer to Q4

stx = 'شب است و شمع و شراب و شیرینی'

Why we cannot find occurrence of stx? 
+ one of the reasons coule be: Although we have seen all of the stx's tokens, we have not seen some of its triples

To tackle this issue:
1. we can replace its probability with the nearest n-gram
2. in case we have not seen a toke like شیرینی in our corpus, we can replace شیرینی with the most similar word that we have in our Corpus,can't we?

#### Answer to Q5

In [153]:
lst = ["چون مشک سیه بود مرا هر دو بنا","گر خورد سوگند هم آن","زانک نفس آشفته تر گردد از","ازین زشت تر در جهان رنگ"]
data_frames = []
for sent in lst:
    data_frames.append(question5handler(sent,d2)) 


The sentence: چون مشک سیه بود مرا هر دو بنا can be completed with bigrams in the following way:

First way for completing: چون مشک سیه بود مرا هر دو بنا گوش
Second way for completing: چون مشک سیه بود مرا هر دو بنا و
The sentence: گر خورد سوگند هم آن can be completed with bigrams in the following way:

First way for completing: گر خورد سوگند هم آن را
Second way for completing: گر خورد سوگند هم آن که
The sentence: زانک نفس آشفته تر گردد از can be completed with bigrams in the following way:

First way for completing: زانک نفس آشفته تر گردد از آن
Second way for completing: زانک نفس آشفته تر گردد از تو
The sentence: ازین زشت تر در جهان رنگ can be completed with bigrams in the following way:

First way for completing: ازین زشت تر در جهان رنگ و
Second way for completing: ازین زشت تر در جهان رنگ رنگ


In [155]:
data_frames[0].head()

,terms,counts,freq,freq_index
15881,بنا گوش,7,0.000006,0.099371
27053,بنا و,5,0.000004,0.120908
33723,بنا کرده,4,0.000004,0.120574
56693,بنا را,3,0.000003,0.152024
56696,بنا کرد,3,0.000003,0.152032


In [156]:
data_frames[1].head()

,terms,counts,freq,freq_index
75,آن را,324,0.000290,0.022010
76,آن که,322,0.000288,0.022162
202,آن یکی,180,0.000161,0.032661
255,آن به,159,0.000142,0.036382
357,آن کس,126,0.000113,0.040319


In [157]:
data_frames[2].head()

,terms,counts,freq,freq_index
0,از آن,1394,0.001246,0.001246
7,از تو,779,0.000696,0.005570
16,از پی,612,0.000547,0.009299
31,از این,480,0.000429,0.013729
34,از بهر,463,0.000414,0.014485


In [158]:
data_frames[3].head()

,terms,counts,freq,freq_index
469,رنگ و,106,0.000095,0.044531
5264,رنگ رنگ,18,0.000016,0.084708
5814,رنگ از,17,0.000015,0.088360
8094,رنگ او,13,0.000012,0.094062
11134,رنگ است,10,0.000009,0.099528


# Further Development notes

1. df['id'] column is redundent (solved)
2. optimize question5hendler function

# Thanks for your attention
# Any feedback or issue is very welcome